# Install and Import Packages

In [5]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to c:\users\tpeiq\appdata\local\temp\pip-req-build-p1jzymtw
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 2608e27137810d6803565e473edb81f4c47ae866
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to c:\users\tpeiq\appdata\local\temp\pip-install-4o2eyclf\elegantrl_aeaa92b229a140c28efd4a8e1425f16e
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 8ea76afc3e7f1564ae9f0e69e70254116d575fe9
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git 'C:\Users\tpeiq\AppData\Local\Temp\pip-req-build-p1jzymtw'
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git 'C:\Users\tpeiq\AppData\Local\Temp\pip-install-4o2eyclf\elegantrl_aeaa92b229a140c28efd4a8e1425f16e'


In [1]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
import itertools
check_and_make_directories([TRAINED_MODEL_DIR])



In [2]:
from datetime import datetime, timedelta

# Set the end date as today
TRADE_END_DATE = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

# Set the trade start date as 3 months before the end date
TRADE_START_DATE = (datetime.today() - timedelta(days=90)).strftime('%Y-%m-%d')

# Set the train end date as the trade start date
TRAIN_END_DATE = TRADE_START_DATE

# Set the train start date as 5 years before the train end date
TRAIN_START_DATE = (datetime.strptime(TRAIN_END_DATE, '%Y-%m-%d') - timedelta(days=5*365)).strftime('%Y-%m-%d')

print("TRAIN_START_DATE:", TRAIN_START_DATE)
print("TRAIN_END_DATE:", TRAIN_END_DATE)
print("TRADE_START_DATE:", TRADE_START_DATE)
print("TRADE_END_DATE:", TRADE_END_DATE)

TRAIN_START_DATE: 2020-03-24
TRAIN_END_DATE: 2025-03-23
TRADE_START_DATE: 2025-03-23
TRADE_END_DATE: 2025-06-20


In [13]:
symbols = ['aapl', 'amd', 'amzn', 'cat', 'crwd', 'googl', 'gs', 'hd', 'ibm',
       'intc', 'meta', 'msft', 'nvda', 'pypl', 't', 'tsla', 'v']

In [14]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRADE_END_DATE,
                                ticker_list = symbols).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (22389, 8)


In [15]:
df_raw.head()

Price,date,close,high,low,open,volume,tic,day
0,2020-03-24,59.834133,60.030445,56.785229,57.284492,287531200,aapl,1
1,2020-03-24,46.220001,46.810001,43.990002,44.040001,106794200,amd,1
2,2020-03-24,97.004997,97.750000,95.016998,97.574997,142942000,amzn,1
3,2020-03-24,90.671463,91.288826,85.151009,85.947314,7417100,cat,1
4,2020-03-24,57.330002,58.650002,54.279999,54.919998,8088000,crwd,1


# Preprocess Data

In [16]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (1316, 8)
Successfully added vix


Successfully added turbulence index


In [17]:
processed.tic.unique()

array(['aapl', 'amd', 'amzn', 'cat', 'crwd', 'googl', 'gs', 'hd', 'ibm',
       'intc', 'meta', 'msft', 'nvda', 'pypl', 't', 'tsla', 'v'],
      dtype=object)

In [18]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [19]:
processed_full.tic.unique()

array(['aapl', 'amd', 'amzn', 'cat', 'crwd', 'googl', 'gs', 'hd', 'ibm',
       'intc', 'meta', 'msft', 'nvda', 'pypl', 't', 'tsla', 'v'],
      dtype=object)

# Save the Data

In [20]:
# Split the data
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

21352
1020


In [21]:
train_path = './data/train_data.csv'
trade_path = './data/trade_data.csv'

with open(train_path, 'w', encoding = 'utf-8-sig') as f:
  train.to_csv(f)

with open(trade_path, 'w', encoding = 'utf-8-sig') as f:
  trade.to_csv(f)

# Load the Data from Part 1

In [22]:
train = pd.read_csv(train_path)

train = train.set_index(train.columns[0])
train.index.names = ['']

## Setup Environment

In [23]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 17, State Space: 171


In [24]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [25]:
train

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
,,,,,,,,,,,,,,,,,,
0,2020-03-24,aapl,59.834133,60.030445,56.785229,57.284492,287531200.0,1.0,0.000000,60.135459,59.203206,0.000000,66.666667,100.000000,59.834133,59.834133,61.669998,0.000000
0,2020-03-24,amd,46.220001,46.810001,43.990002,44.040001,106794200.0,1.0,0.000000,60.135459,59.203206,0.000000,66.666667,100.000000,46.220001,46.220001,61.669998,0.000000
0,2020-03-24,amzn,97.004997,97.750000,95.016998,97.574997,142942000.0,1.0,0.000000,60.135459,59.203206,0.000000,66.666667,100.000000,97.004997,97.004997,61.669998,0.000000
0,2020-03-24,cat,90.671463,91.288826,85.151009,85.947314,7417100.0,1.0,0.000000,60.135459,59.203206,0.000000,66.666667,100.000000,90.671463,90.671463,61.669998,0.000000
0,2020-03-24,crwd,57.330002,58.650002,54.279999,54.919998,8088000.0,1.0,0.000000,60.135459,59.203206,0.000000,66.666667,100.000000,57.330002,57.330002,61.669998,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,2025-03-21,nvda,117.691826,117.981807,115.411985,116.931884,266498500.0,4.0,-3.110946,131.012237,105.089311,46.086341,-52.661001,19.378736,123.848561,129.111434,19.280001,10.312175
1255,2025-03-21,pypl,70.190002,70.410004,68.550003,68.620003,14603800.0,4.0,-2.355786,74.021382,65.723621,41.625566,-48.632552,17.729651,72.320000,79.724250,19.280001,10.312175
1255,2025-03-21,t,26.736223,26.924226,26.360213,26.528428,91268600.0,4.0,0.410593,27.393626,25.446532,61.034385,74.791767,17.571132,26.073260,24.343506,19.280001,10.312175


### Environment for Training

In [26]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# Train Agent

In [28]:
agent = DRLAgent(env = env_train)

# Set the corresponding values to 'True' for the algorithms that you want to use
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = True
if_using_sac = True

## A2C Model

The code above and below provide a healthy framework to add different agent types for training within the same notebook. For this I'll only be training an a2c model, but may add ddpg, ppo, and td3 in later iterations.

In [29]:
model_a2c = agent.get_model("a2c")
model_ppo = agent.get_model('ppo')

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device
{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 64}
Using cuda device
Logging to results/a2c
Logging to results/ppo


c:\Users\tpeiq\anaconda3\envs\pyspark310\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
c:\Users\tpeiq\anaconda3\envs\pyspark310\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `e

In [30]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3070 Laptop GPU


In [31]:
trained_a2c = agent.train_model(model=model_a2c,
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=50000) if if_using_ppo else None

--------------------------------------
| time/                 |            |
|    fps                | 112        |
|    iterations         | 100        |
|    time_elapsed       | 4          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -24        |
|    explained_variance | 0.192      |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | 55.4       |
|    reward             | 1.3259981  |
|    reward_max         | 4.2358     |
|    reward_mean        | 1.17481    |
|    reward_min         | -2.0534625 |
|    std                | 0.993      |
|    value_loss         | 11         |
--------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 114        |
|    iterations         | 200        |
|    time_elapsed       | 8          |
|    total_timesteps    | 1000       |
| train/                |

In [32]:
trained_a2c.save(TRAINED_MODEL_DIR + "/agent_a2c") if if_using_a2c else None
trained_ppo.save(TRAINED_MODEL_DIR + "/agent_ppo") if if_using_ppo else None

In [33]:

if_using_sac = True

model_sac = agent.get_model("sac")


if if_using_sac:
    tmp_path = RESULTS_DIR + '/sac'
    # Only use basic loggers to avoid rollout_buffer errors
    new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
    model_sac.set_logger(new_logger_sac)





{'batch_size': 64, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cuda device
Logging to results/sac


In [34]:
trained_sac = agent.train_model(model=model_sac, tb_log_name='sac', total_timesteps=50000) if if_using_sac else None

Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging 

In [35]:
trained_sac.save(TRAINED_MODEL_DIR + "/agent_sac") if if_using_sac else None